In [1]:
import numpy as np
import math, copy

# Load sudokus
sudoku = np.load("data/very_easy_puzzle.npy")
print("very_easy_puzzle.npy has been loaded into the variable sudoku")
print(f"sudoku.shape: {sudoku.shape}, sudoku[0].shape: {sudoku[0].shape}, sudoku.dtype: {sudoku.dtype}")

# Load solutions for demonstration
solutions = np.load("data/very_easy_solution.npy")
print()

# Print the first 9x9 sudoku...
print("First sudoku:")
print(sudoku[0], "\n")

# ...and its solution
print("Solution of first sudoku:")
print(solutions[0])

very_easy_puzzle.npy has been loaded into the variable sudoku
sudoku.shape: (15, 9, 9), sudoku[0].shape: (9, 9), sudoku.dtype: int8

First sudoku:
[[1 0 4 3 8 2 9 5 6]
 [2 0 5 4 6 7 1 3 8]
 [3 8 6 9 5 1 4 0 2]
 [4 6 1 5 2 3 8 9 7]
 [7 3 8 1 4 9 6 2 5]
 [9 5 2 8 7 6 3 1 4]
 [5 2 9 6 3 4 7 8 1]
 [6 0 7 2 9 8 5 4 3]
 [8 4 3 0 1 5 2 6 9]] 

Solution of first sudoku:
[[1 7 4 3 8 2 9 5 6]
 [2 9 5 4 6 7 1 3 8]
 [3 8 6 9 5 1 4 7 2]
 [4 6 1 5 2 3 8 9 7]
 [7 3 8 1 4 9 6 2 5]
 [9 5 2 8 7 6 3 1 4]
 [5 2 9 6 3 4 7 8 1]
 [6 1 7 2 9 8 5 4 3]
 [8 4 3 7 1 5 2 6 9]]


## Part One
You should write all of your code for solving sudokus below this cell.

You must include a function called `sudoku_solver(sudoku)` which takes one sudoku puzzle (a 9x9 NumPy array) as input, and returns the solved sudoku as another 9x9 NumPy array. This is the function which will be tested. 

In [2]:
class Queue:
    def __init__(self):
        self.items = []

    def isEmpty(self):
         return len(self.items) == 0

    def enqueue(self, newItem):
        self.items.append(newItem)

    def dequeue(self):
        return self.items.pop(0)

    def size(self):
        return len(self.items)

In [3]:
def insert_value(sudoku: object, variable: tuple, value: int, return_copy: bool = False) -> None or object:
    """
    Inserts the value of a variable into the sudoku board at the variables position.

    Input
        sudoku: 9x9 numpy array
            Empty cells are designated by 0.

        variable: tuple of length 2
            (rowIdx, colIdx) of the variable's position in the board.

        value: int
            value is between 0-9

        return_copy: bool
            false (default) if this change should be written to the current board. 
            
    Output
        object | none:
            returns 9x9 numpy array if return_copy = true.
    """

    # get row and column for insertion
    row_idx, col_idx = variable

    # return a copy of the board
    if return_copy:
        new_board = np.copy(sudoku)
        new_board[row_idx][col_idx] = value
        return new_board
        
    # or insert by reference
    else:
        sudoku[row_idx][col_idx] = value

In [4]:
def any_duplicates(a_array: object) -> bool:
    """
    Given a 1d numpy array of integers, return whether there is a repeated value that isn't zero.
    """
    # set of already added numbers
    seen = set()
    
    for element in list(a_array):

        # if element is duplicate and is between 1-9 (not 0)
        if element in seen and element != 0:
            return True

        seen.add(element)
        
    return False

In [5]:
def is_illegal_board(sudoku: object) -> bool:
    """
    Takes an unfinished board and determines whether it is an illegal board.

    Input
        sudoku: 9x9 numpy array
            Empty cells are designated by 0.
            
    Ouput
        boolean:
            indicates whether current sudoku board is illegal.
    """
    for i in range(9):

        # 1 & 2. if 1-9 is repeated in rows or column -> illegal
        row = sudoku[i]
        column = sudoku[:,i]
        if any_duplicates(row[row != 0]) or any_duplicates(column[column != 0]):
            return True

    for rowIdx in range(0,9,3):
        for colIdx in range(0,9,3):

            # 3. if 1-9 is repeated in subgrid -> illegal
            subgrid = sudoku[rowIdx: rowIdx + 3][:, colIdx: colIdx + 3].flatten()
            if any_duplicates(subgrid[subgrid != 0]):
                return True
    
    # else: valid
    return False

In [6]:
def is_solved(sudoku: object) -> bool:
    """
    Return whether the current sudoku board is solved.

    Input
        sudoku: 9x9 numpy array
            Empty cells are designated by 0.

    Output
        boolean:
            indicating whether the current state of the board is solved.
    """

    # required values: 1-9
    required_values = set(range(1,10)) # unordered

    for i in range(9):

        # 1 & 2. is 1-9 is not in every row and column, not solved
        if set(sudoku[i]) != required_values or set(sudoku[:,i]) != required_values:
            return False

    for rowIdx in range(0,9,3):
        for colIdx in range(0,9,3):

            # 3. if 1-9 is not in every subgrid, not solved
            if set(sudoku[rowIdx: rowIdx + 3][:, colIdx: colIdx + 3].flatten()) != required_values:
                return False
    
    # else: solved
    return True

In [7]:
# test above function
for solved_board in solutions:
    print(is_solved(solved_board))

for unsolved_board in sudoku:
    print(is_solved(unsolved_board))

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False


In [8]:
def satisfies_constraint(sudoku: object, variable: tuple, value: int) -> bool:
    """
    Given a new possible variable value, determine whether this would be a legal move.

    Input
        sudoku: 9x9 numpy array
            Empty cells are designated by 0.

        variable:
            tuple containing the position of the variable (row_idx, col_idx).
            
        value:
            value we're testing is allowed to be at this position

    Output
        boolean:
            indicating whether the value is legal at that position.
    """
    row_idx, col_idx = variable

    # Extract row and column of variable
    row, column = list(sudoku[row_idx]), list(sudoku[:,col_idx])

    # Extract the rows of the subgrid
    start_subgrid_row_idx = 3 * (row_idx // 3)
    subgrid = sudoku[start_subgrid_row_idx : start_subgrid_row_idx + 3]

    # Extract the columns of the subgrid
    start_subgrid_col_idx = 3 * (col_idx // 3)
    subgrid = list(subgrid[:, start_subgrid_col_idx : start_subgrid_col_idx + 3].flatten())

    # If any values repeat then -> not valid
    if value in set(row + column + subgrid):
        return False
    else:
        return True

In [9]:
# test on the first puzzle
test1_sudoku = sudoku[0]

# we will be testing on the variable (0,1)
print(test1_sudoku)
variable = (0,1)
value = 7

print(satisfies_constraint(test1_sudoku, variable, value))
# returns True - expected value. Valid Sudoku board

value = 4
print(satisfies_constraint(test1_sudoku, variable, value))
# returns False - expected value. Invalid sudoku board

[[1 0 4 3 8 2 9 5 6]
 [2 0 5 4 6 7 1 3 8]
 [3 8 6 9 5 1 4 0 2]
 [4 6 1 5 2 3 8 9 7]
 [7 3 8 1 4 9 6 2 5]
 [9 5 2 8 7 6 3 1 4]
 [5 2 9 6 3 4 7 8 1]
 [6 0 7 2 9 8 5 4 3]
 [8 4 3 0 1 5 2 6 9]]
True
False


In [10]:
def find_empty(sudoku):

    for row_idx in range(9): 
        for col_idx in range(9):

            if sudoku[row_idx][col_idx] == 0:
                # Position is stored in the form of a tuple (row, column) / matrix notation
                postion = (row_idx, col_idx)
                return postion
    
    return None

In [11]:
def get_variables(sudoku: object) -> list:
    """
    Returns a list of the empty spaces (0s) in a given sudoku board.
    Input
        sudoku: 9x9 numpy array
            Empty cells are designated by 0.

    Output
        variables:
            list of tuples of length 2. Each tuple containing the position (row, col) of variables in the sudoku board.
            
    """
    # Initialise variables: each var will be the position of an empty space. Denoted by 0. 
    variables = []

    # 9 x 9 sudoku board = 81 iterations
    for row_idx in range(9): 
        for col_idx in range(9):

            if sudoku[row_idx][col_idx] == 0:
                # Position is stored in the form of a tuple (row, column) / matrix notation
                postion = (row_idx, col_idx)
                variables.append(postion)

    return variables

In [12]:
# test on the second sudoku 
print(sudoku[1], "\n")

 # returns the positions of all empty spaces: returns expected value
test_vars = get_variables(sudoku[1])
print(test_vars)

[[0 9 3 1 5 2 6 0 8]
 [8 6 2 7 0 3 1 9 5]
 [1 5 7 9 8 6 3 2 4]
 [9 7 8 4 2 1 0 3 6]
 [5 0 6 8 3 9 4 1 7]
 [3 4 1 5 6 7 2 8 9]
 [6 1 4 2 7 8 9 5 3]
 [7 3 9 6 1 5 8 4 2]
 [2 8 5 3 9 4 7 6 1]] 

[(0, 0), (0, 7), (1, 4), (3, 6), (4, 1)]


In [13]:
set([1, 2, 3]).difference(set([1, 2]))


{3}

In [14]:
def get_domains(sudoku: object, variables: list) -> list or None:
        """
        Returns the list of legal values a variable can take according to a constraint, for every variable.

        Input
            sudoku: 9x9 numpy array
                Empty cells are designated by 0.
            variables:
                list of tuples of length 2. Each tuple containing the position (row, col) of variables in the sudoku board.

        Output
            domain: dictionary of lists
                Each nested list contains the legal values that a variable can take.
        """

        domains = dict()

        # Extract row and column of variable
        for var_i in variables:

            # Extract rows and columns
            row_idx, col_idx = var_i
            row, column = list(sudoku[row_idx]), list(sudoku[:,col_idx])

            # Extract the rows of the subgrid
            start_subgrid_row_idx = 3 * (row_idx // 3)
            subgrid = sudoku[start_subgrid_row_idx : start_subgrid_row_idx + 3]

            # Extract the columns of the subgrid
            start_subgrid_col_idx = 3 * (col_idx // 3)
            subgrid = list(subgrid[:, start_subgrid_col_idx : start_subgrid_col_idx + 3].flatten())

            # If any values repeat then -> not valid
            possible_values = set(list(range(1, 10)))
            existing_values = set(row + column + subgrid)
            
            domains[var_i] = list(possible_values.difference(existing_values))

        # if not update operation -> return new copy
        return domains

In [15]:
print(get_domains(sudoku[1], test_vars))
print(sudoku[1])

{(0, 0): [4], (0, 7): [7], (1, 4): [4], (3, 6): [5], (4, 1): [2]}
[[0 9 3 1 5 2 6 0 8]
 [8 6 2 7 0 3 1 9 5]
 [1 5 7 9 8 6 3 2 4]
 [9 7 8 4 2 1 0 3 6]
 [5 0 6 8 3 9 4 1 7]
 [3 4 1 5 6 7 2 8 9]
 [6 1 4 2 7 8 9 5 3]
 [7 3 9 6 1 5 8 4 2]
 [2 8 5 3 9 4 7 6 1]]


In [16]:
def get_neighbour_nodes(variables: list) -> list:
    """
    Given a list of variables -> returns each variable's neighbours.
     
    Input:
        variables:
            list of tuples of length 2. Each tuple containing the position (row, col) of variables in the sudoku board.

    Output:
        neighbours:
            list of sets for each variable containing the indices of the neighbours for the variable.
            var_i_idx = var_i_neighbours_idx
    """
    neighbours = dict()
    for var_i in variables:

        # variable's neighbours is stored in a set
        var_i_neighbours = set()
        for var_j in variables:

            # node cannot be a neighbour of itself
            if var_i != var_j:
                
                # get rows and columns
                var_i_row, var_i_col = var_i 
                var_j_row, var_j_col = var_j

                # 1. If variables in same row -> they are related.
                if var_i_row == var_j_row:
                    var_i_neighbours.add(var_j)
                    continue 

                # 2. If variables in same column -> they are related.
                if var_i_col == var_j_col:
                    var_i_neighbours.add(var_j)
                    continue

                # Determine the subgrid position of i
                var_i_subgrid_row_idx = math.ceil( (var_i_row + 1) / 3) - 1
                var_i_subgrid_col_idx = math.ceil( (var_i_col + 1) / 3) - 1
                var_i_subgrid = (var_i_subgrid_row_idx, var_i_subgrid_col_idx)

                # ... and j
                var_j_subgrid_row_idx = math.ceil( (var_j_row + 1) / 3) - 1
                var_j_subgrid_col_idx = math.ceil( (var_j_col + 1) / 3) - 1
                var_j_subgrid = (var_j_subgrid_row_idx, var_j_subgrid_col_idx)

                # 3. If variables in same subgrid -> they are related.
                if var_i_subgrid == var_j_subgrid:
                    var_i_neighbours.add(var_j)
                    continue

        neighbours[var_i] = var_i_neighbours

    return neighbours

In [17]:
# three variables in the same row
var1, var2, var3  = (1,2), (1,4), (1,8)

# three variables in the same column
var4, var5, var6 = (2, 0), (2, 5), (2, 8)

variables = [var1, var2, var3, var4, var5, var6]

# vars 1 and 4 are in the same subgrid, so are vars 2 and 5 and vars 5 and 6
neighbours = get_neighbour_nodes(variables)
print(neighbours)

{(1, 2): {(1, 8), (2, 0), (1, 4)}, (1, 4): {(1, 8), (1, 2), (2, 5)}, (1, 8): {(1, 2), (1, 4), (2, 8)}, (2, 0): {(1, 2), (2, 5), (2, 8)}, (2, 5): {(2, 0), (1, 4), (2, 8)}, (2, 8): {(2, 5), (1, 8), (2, 0)}}


In [18]:
# find the domains of the variables for the vars found in the second test
test_domains = get_domains(sudoku[1], test_vars)

print("\nPuzzle 1:\n", sudoku[1], "\n\nSolution:\n" ,solutions[1], "\n")
print("Domain for puzzle 1:\n", test_domains, "\n")


Puzzle 1:
 [[0 9 3 1 5 2 6 0 8]
 [8 6 2 7 0 3 1 9 5]
 [1 5 7 9 8 6 3 2 4]
 [9 7 8 4 2 1 0 3 6]
 [5 0 6 8 3 9 4 1 7]
 [3 4 1 5 6 7 2 8 9]
 [6 1 4 2 7 8 9 5 3]
 [7 3 9 6 1 5 8 4 2]
 [2 8 5 3 9 4 7 6 1]] 

Solution:
 [[4 9 3 1 5 2 6 7 8]
 [8 6 2 7 4 3 1 9 5]
 [1 5 7 9 8 6 3 2 4]
 [9 7 8 4 2 1 5 3 6]
 [5 2 6 8 3 9 4 1 7]
 [3 4 1 5 6 7 2 8 9]
 [6 1 4 2 7 8 9 5 3]
 [7 3 9 6 1 5 8 4 2]
 [2 8 5 3 9 4 7 6 1]] 

Domain for puzzle 1:
 {(0, 0): [4], (0, 7): [7], (1, 4): [4], (3, 6): [5], (4, 1): [2]} 



In [19]:
def revise_domain(sudoku: object, variables: list, domains: dict, var_i: int, var_j: int) -> bool:
    """
    Given the index of a variables i and j, remove values from the domain of i which form no relation
     with the values from the domain of var j. 

    Input:
        sudoku: 9x9 numpy array
            Empty cells are designated by 0.

        variables: list of tuples of length 2.
            Each tuple containing the position (row, col) of variables in the sudoku board.

        var_i_idx, var_j_idx: positive integer including 0.
            var_i's domain is what we are revising against var_j's domain
        
        domains: list of lists.
            Each sublist contains the allowable values for a variable. var_i_idx = dom_of_var_i_idx.
    
    Output:
        revised: boolean indicating whether the domain has been revised.
        * changes to domain are acheived by reference to the domain argument.
    """

    # flag indicating revision
    revised = False

    # 1. Repeat for every value in the domain of var i
    for var_i_domain_val in list(domains[var_i]):
        
        # Insert var i domain value into copy of board
        new_board = insert_value(sudoku, var_i, var_i_domain_val, True)

        # 2. Create a list of bools for which the domain values of vars i and j satisfy the binary constraint.
        domain_ij_formed_relation = map(lambda var_j_domain_val:
            satisfies_constraint(new_board, var_j, var_j_domain_val), domains[var_j]
        )

        # 3. If no value of domain var j worked for a value of var i -> remove the value from domain of var i
        if not any(domain_ij_formed_relation):
            domains[var_i].remove(var_i_domain_val)
            revised = True     
    
    return revised

In [20]:
def arc_consistency(sudoku: object, variables: list, domains: dict) -> bool:
    """
    Makes arcs between all arcs in the problem consistent by removing all domains of all inconsistent domain values.
    Returns a boolean indicating false if any of these domains are now empty.
    Input
        sudoku: 9x9 numpy array
            Empty cells are designated by 0.

        variables: list of tuples of length 2.
            Each tuple containing the position (row, col) of variables in the sudoku board.

        neighbours:
            list of sets for each variable containing the indices of the neighbours for the variable.
            var_i_idx = var_i_neighbours_idx
        
        domain: list of lists.
            Each sublist contains the allowable values for a variable. var_i_idx = dom_of_var_i_idx.

    Output
        boolean:
            True if arcs are now consistent. False if not / sudoku is not solvable.
    """

    # Queue of all the arcs to revise the domain against
    agenda = Queue()

    # 0. Get the neighbours of each variable
    neighbours = get_neighbour_nodes(variables)

    # 1. Add all arcs (relation between neighbours) between variables to the agenda.
    for var_i, var_i_neighbours in neighbours.items():

        # parse variable index from string -> integer
        for var_j in var_i_neighbours:

                # vars represented by their indicies in the 'variables' list
                arc = (var_i, var_j) 
                agenda.enqueue(arc)

    # 2. Repeat until the arc agenda is empty:
    while agenda.size() != 0:

        # 2.1. Dequeue  (Xi, Xj) from the agenda.
        (var_i, var_j) = agenda.dequeue()

        # 2.2. For every value of the doamin of var_i there must be a value of var_j that satisfies the constraint.
        #     Revise the domain of var i so this is true.
        if revise_domain(sudoku, variables, domains, var_i, var_j):
            
            # 2.3. If this new domain is empty, there are no legal values for var i. 
            if len(domains[var_i]) == 0:

                # Sudoku is unsolvable if a variable has no domain.
                return False
            
            # 2.4. Change of domain -> we must revise i's neighbours with respect to i's' new domain.          
            for var_k in variables:

                # Let var_k = the neighbours of i except j (since var_i's domain is now consistent with j's.)
                if var_k != var_i and var_k != var_j:

                    arc = (var_k, var_i)
                    agenda.enqueue(arc)

    return True

## Recursive Backtracking solution for harder sudoku
1. Make assignment by Minimal Remaining Value
2. Smart configure variables, domain and neighbours
3. Repeat AC-3

In [21]:
def most_constrained_variable(domains: dict) -> int:
    """
    Returns the index of the variable with the smallest domain. (Minimum Remaining value)
    Input:
        domain: list of lists.
                Each sublist contains the allowable values for a variable. var_i_idx = dom_of_var_i_idx.
    Output:
        integer:
            Index of the variable with the smallest domain. Returns -1 if there are no values left
    """
    if len(domains) == 0:
        return None
    else:
        return min(domains, key = lambda i: len(domains[i]))

In [22]:
def backtrack_search2(sudoku: object, revised_domains = None) -> bool:

    """
    A depth first search which backtracks at dead-ends and returns whether a solution could be found

    Input:
        sudoku : 9x9 numpy array
            Empty cells are designated by 0.

        revised_domains: dictionary of lists
            Each nested list contains the legal values that a variable can take.
    
    Output:
        boolean indicating whether the sudoku can be solved
        
    """

    variables = get_variables(sudoku)
    if revised_domains != None:
        domains = revised_domains
    else:
        domains = get_domains(sudoku, variables)
    
    var_s = most_constrained_variable(domains)
    if var_s == None:
        return True

    for possible_var_s_value in domains[var_s]:

        # assign the variable to the board
        insert_value(sudoku, var_s, possible_var_s_value, False)
        
        # ends recursion if solved
        if backtrack_search2(sudoku, None):
            return True

        # resets assignment -> 0 means empty space
        insert_value(sudoku, var_s, 0)
    
    # when no values in var_s domain worked
    return False

In [23]:
def sudoku_solver(sudoku: object) -> object:
    """
    Solves a Sudoku puzzle and returns its unique solution.

    Input
        sudoku : 9x9 numpy array
            Empty cells are designated by 0.

    Output
        9x9 numpy array of integers
            It contains the solution, if there is one. If there is no solution, all array entries should be -1.
    """

    # 9 x 9 board: all array entries should be -1
    unsolvable_board = np.array([np.array([-1] * 9) ] * 9)

    # 0. Pre-emptive check that the board is valid
    if is_illegal_board(sudoku):
        return unsolvable_board
    
    # 1. Locate the variables on the sudoku board.
    variables = get_variables(sudoku)

    # 2. Find the legal values these variables can take -> unary constraint creates domain.
    domains = get_domains(sudoku, variables)

    # 3. Pre-emptive check that there is no variables with no domain -> no solution
    if any(map(lambda ith_domain: len(ith_domain) == 0, domains)): return unsolvable_board

    # 4. Make arcs between variables consistent:
    arcs_are_consistent = arc_consistency(sudoku, variables, domains)

    # 5. If arcs aren't consistent, board is unsolvable
    if not arcs_are_consistent: return unsolvable_board

    # 6a. If all domains are of length -> solution is found
    if  all(map(lambda ith_domain: len(ith_domain) == 1, domains.values())):

        # Construct the final solution:
        for var_i, var_i_domain in domains.items():
            insert_value(sudoku, var_i, var_i_domain[0])
    
    # 6b. If more than one value in a variables domain -> recursively make assignments with DFS
    else:

        solved = backtrack_search2(sudoku, domains)
        if not solved:
            return unsolvable_board

    return sudoku

All of your code must go above this cell. You may add additional cells into the notebook if you wish, but do not duplicate or copy/paste cells as this can interfere with the grading script.

### Testing Details
There are four difficulties of sudoku provided: very easy, easy, medium, and hard. There are 15 sample sudokus in each category, with solutions as well. Difficulty was determined using reference solvers, but your code may vary; it is conceivable that your code will find some sudokus much easier or harder within a given category, or even between categories.

*All categories that are easy and above will contain* ***invalid initial states***, that is, sudoku puzzles with no solution. In this case, your function should return a 9x9 NumPy array whose values are all equal to -1.

When we test your code, we will firstly test it on the *same* very easy puzzles that you have been given. Then we will test it on additional *hidden* sudokus from each difficulty in turn, easy and up – many more than 15 of each. Grades are awarded based on whether your code can solve the puzzles. For high grades on the hard puzzles, execution time will also be a factor. 

All puzzles must take under 20 seconds each on the test machine to count as successful – if it takes longer, it will timeout. Note that this is a maximum, not a goal. Higher grades require better performance on harder puzzles. As a very rough benchmark, you should be aiming for an average of under a second per puzzle. Hardware varies, but all tests will take place on the same modern desktop machine. Our ‘standard constraint satisfaction’ implementation takes about 0.001 seconds per puzzle for the very easy category, but struggles to solve some of the hard puzzles within the time limit.

***The hard sudokus are labelled as hard for a reason.*** We expect most submissions will not be able to solve them in a reasonable length of time. Use the stop button (■) on the toolbar if you need to terminate your code because it is taking too long.

The best way to improve the performance of your code is through a detailed understanding and smart choice of AI algorithms and data structures. This assignment is ***not*** meant to test your ability to write multi-threaded code or any other kind of high-performance code optimisations. 

#### Test Cell
The following code will run your solution over the provided sudoku puzzles. To enable it, set the constant `SKIP_TESTS` to `False`. If you fail any tests of one difficulty, the code will stop, but you can modify this behaviour if you like.

**IMPORTANT**: you must set `SKIP_TESTS` back to `True` before submitting this file!

In [24]:
SKIP_TESTS = True

def tests():
    import time
    difficulties = ['very_easy', 'easy', 'medium', 'hard']

    for difficulty in difficulties:
        print(f"Testing {difficulty} sudokus")
        
        sudokus = np.load(f"data/{difficulty}_puzzle.npy")
        solutions = np.load(f"data/{difficulty}_solution.npy")
        
        count = 0
        for i in range(len(sudokus)):
            sudoku = sudokus[i].copy()
            print(f"This is {difficulty} sudoku number", i)
            print(sudoku)
            
            start_time = time.process_time()
            your_solution = sudoku_solver(sudoku)
            end_time = time.process_time()
            
            print(f"This is your solution for {difficulty} sudoku number", i)
            print(your_solution)
            
            print("Is your solution correct?")
            if np.array_equal(your_solution, solutions[i]):
                print("Yes! Correct solution.")
                count += 1
            else:
                print("No, the correct solution is:")
                print(solutions[i])
            
            print("This sudoku took", end_time-start_time, "seconds to solve.\n")

        print(f"{count}/{len(sudokus)} {difficulty} sudokus correct")
        if count < len(sudokus):
            break
            
if not SKIP_TESTS:
    tests()

Testing very_easy sudokus
This is very_easy sudoku number 0
[[1 0 4 3 8 2 9 5 6]
 [2 0 5 4 6 7 1 3 8]
 [3 8 6 9 5 1 4 0 2]
 [4 6 1 5 2 3 8 9 7]
 [7 3 8 1 4 9 6 2 5]
 [9 5 2 8 7 6 3 1 4]
 [5 2 9 6 3 4 7 8 1]
 [6 0 7 2 9 8 5 4 3]
 [8 4 3 0 1 5 2 6 9]]
This is your solution for very_easy sudoku number 0
[[1 7 4 3 8 2 9 5 6]
 [2 9 5 4 6 7 1 3 8]
 [3 8 6 9 5 1 4 7 2]
 [4 6 1 5 2 3 8 9 7]
 [7 3 8 1 4 9 6 2 5]
 [9 5 2 8 7 6 3 1 4]
 [5 2 9 6 3 4 7 8 1]
 [6 1 7 2 9 8 5 4 3]
 [8 4 3 7 1 5 2 6 9]]
Is your solution correct?
Yes! Correct solution.
This sudoku took 0.0 seconds to solve.

This is very_easy sudoku number 1
[[0 9 3 1 5 2 6 0 8]
 [8 6 2 7 0 3 1 9 5]
 [1 5 7 9 8 6 3 2 4]
 [9 7 8 4 2 1 0 3 6]
 [5 0 6 8 3 9 4 1 7]
 [3 4 1 5 6 7 2 8 9]
 [6 1 4 2 7 8 9 5 3]
 [7 3 9 6 1 5 8 4 2]
 [2 8 5 3 9 4 7 6 1]]
This is your solution for very_easy sudoku number 1
[[4 9 3 1 5 2 6 7 8]
 [8 6 2 7 4 3 1 9 5]
 [1 5 7 9 8 6 3 2 4]
 [9 7 8 4 2 1 5 3 6]
 [5 2 6 8 3 9 4 1 7]
 [3 4 1 5 6 7 2 8 9]
 [6 1 4 2 7 8 9 5

## Submission Test
The following cell tests if your notebook is ready for submission. **You must not skip this step!**

Restart the kernel and run the entire notebook (Kernel → Restart & Run All). Now look at the output of the cell below. 

*If there is no output, then your submission is not ready.* Either your code is still running (did you forget to skip tests?) or it caused an error.

As previously mentioned, failing to follow these instructions can result in a grade of zero.

In [25]:
def submission_tests():
    import sys
    import pathlib

    fail = False;

    if not SKIP_TESTS:
        fail = True;
        print("You must set the SKIP_TESTS constant to True in the cell above.")

    p1 = pathlib.Path('./readme.txt')
    p2 = pathlib.Path('./readme.md')
    if not (p1.is_file() or p2.is_file()):
        fail = True;
        print("You must include a separate file called readme.txt or readme.md in your submission.")

    p3 = pathlib.Path('./sudoku.ipynb')
    if not p3.is_file():
        fail = True
        print("This notebook file must be named sudoku.ipynb")

    if "sudoku_solver" not in globals():
        fail = True;
        print("You must include a function called sudoku_solver which accepts a numpy array.")
    else: 
        sudoku = np.load("data/very_easy_puzzle.npy")[0]
        solution = np.load("data/very_easy_solution.npy")[0]

        if not np.array_equal(sudoku_solver(sudoku), solution):
            print("Warning:")
            print("Your sudoku_solver function does not correctly solve the first sudoku.")
            print()
            print("Your assignment is unlikely to get any marks from the autograder. While we will")
            print("try to check it manually to assign some partial credit, we encourage you to ask")
            print("for help on the forum or directly to a tutor.")
            print()
            print("Please use the readme file to explain your code anyway.")

    if fail:
        print()
        sys.stderr.write("Your submission is not ready! Please read and follow the instructions above.")
    else:
        print("All checks passed. When you are ready to submit, upload the notebook and readme file to the")
        print("assignment page, without changing any filenames.")
        print()
        print("If you need to submit multiple files, you can archive them in a .zip file. (No other format.)")
        
submission_tests()

You must set the SKIP_TESTS constant to True in the cell above.



Your submission is not ready! Please read and follow the instructions above.

In [26]:
# This is a TEST CELL. Do not delete or change.